In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
inputs= pd.read_csv("./dataset/inputs.csv",names=['txid','prev_tx_id','prev_tx_pos'])
outputs = pd.read_csv("./dataset/outputs.csv",names=['txid','position','address_id','amount','script_type'])
outputs=outputs.astype({
    'script_type':'uint8',
})
transactions = pd.read_csv("./dataset/transactions.csv",names=['timestamp','block_id','txid','is_coinbase','fee'])
transactions= transactions.astype({
    'is_coinbase':'bool',
})
map=pd.read_csv("./dataset/mapAddr2Ids8708820.csv",names=['hash','address_id'])
map.set_index('address_id')

,hash
address_id,
0,1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa
1,12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX
2,1HLoD9E4SDFFPDiYfNYnkBLQ85Y51J3Zb1
3,1FvzCLoTPGANNjWoUo6jUGuAG3wg1w4YjR
4,15ubicBBWFnvoZLT7GiU2qxjRaKJPdkDMG
...,...
8708816,13dJEWNK55L5Zbh8ZVqpvmpritkEYHD1nE
8708817,1GA1vXcj4UMwcgr7nBftkpxXE6RrBwwm2M
8708818,1DrJbVsNJDweTQgKAwrBNyKuvtFBS6iKNK


# 2.1 Analisi generali dei dati della blockchain

## distribuzione transazioni per blocco
distribuzione del numero di transazioni per blocco (occupazione del blocco), nell’intero
periodo temporale considerato

In [ ]:
tx_distribution=transactions.groupby(['block_id']).count()[['timestamp']]
res=pd.DataFrame(columns=['blocks'])
res.index.name='value'
for (i,g) in tx_distribution.groupby(['timestamp']):
    res.loc[i]=len(g)
plt.figure(figsize=(14,5))
plt.scatter(np.log(res.index),res['blocks'],s=1)
plt.xlabel('numero di transazioni')
plt.ylabel('blocchi')

#TODO : fix display of data


## evoluzione dell'occupazione dei blocchi nel tempo
evoluzione dell'occupazione dei blocchi nel tempo, considerando intervalli temporali di due
mesi.(media dell'occupazione di blocchi su archi temporali di 2 mesi).

In [ ]:
from itertools import pairwise

transactions['date']=pd.to_datetime(transactions['timestamp'],unit='s')
month_transactions=[]
month_group=transactions.groupby([transactions['date'].dt.year,transactions['date'].dt.month])

average_tx=pd.DataFrame(columns=['average_tx'])

for (k1,k2) in pairwise(month_group.groups.keys()):
    months_df=pd.concat([month_group.get_group(k1),month_group.get_group(k2)])
    y,m1=k1
    _,m2=k2
    average_tx.loc[str(y)+'-'+str(m1)+'/'+str(m2)]=months_df.groupby(['block_id']).count()['timestamp'].mean()
average_tx.plot(kind='bar',figsize=(14,5))
# TODO: fix display of data

## ammontare degli ultimi UTXO rimanenti
ammontare totale degli UTXO al momento dell’ultima transazione registrata nella
blockchain considerata.

In [ ]:
spent_utxos=outputs.merge(inputs,how='inner',left_on=['txid','position'],right_on=['prev_tx_id','prev_tx_pos'])
unspent_utxos_amount=sum(outputs.amount)-sum(spent_utxos.amount)
unspent_utxos_amount

## durata media di un UTXO prima di essere speso
distribuzione degli intervalli di tempo che intercorrono tra la transazione che genera un
valore in output (UTXO) e quella che lo consuma, per gli output spesi nel periodo
considerato.

In [ ]:
#a=inputs.merge(transactions[['txid','timestamp']],left_on='prev_tx_id',right_on='txid')
#a=a.merge(transactions[['txid','timestamp']],left_on='txid_x',right_on='txid')[['prev_tx_id','timestamp_x','prev_tx_pos','txid','timestamp_y']]

created_tx_ts=inputs.merge(transactions[['txid','timestamp']],left_on='prev_tx_id',right_on='txid')

spent_tx_ts=inputs.merge(transactions[['txid','timestamp']],left_on='txid',right_on='txid')

average_utxo_dt=pd.DataFrame()

average_utxo_dt['deltatime']=pd.to_timedelta(spent_tx_ts['timestamp']-created_tx_ts['timestamp'],unit='s')
average_utxo_dt=average_utxo_dt.groupby('deltatime',group_keys=True).apply(lambda x: len(x))
# TODO: finish data display

## analisi a piacere


## 

# 2.2 clusterizzazione degli indirizzi di bitcoin


In [2]:
import networkx as nx
"""
considerare le transazioni che hanno un numero di input
maggiore di 1 e associare ad uno stesso utente tutti gli indirizzi utilizzati negli input della stessa
transazione.
"""
#associo ad ogni input l'output speso per  ottenere l'indirizzo corrispondente
tx_addresses=inputs.merge(outputs,how='inner',left_on=['prev_tx_id','prev_tx_pos'],right_on=['txid','position'])[['prev_tx_id','address_id']]
#tx_addresses=tx_addresses.merge(map)
tx_addresses=tx_addresses.groupby(['prev_tx_id'])
print('done 1')


done 1


In [36]:
from itertools import islice
from collections import defaultdict
g=nx.DiGraph()
#TODO : fix dataset
for k,group in islice(tx_addresses,10000):
    if(len(group)<=1):
        pass
    a=group['address_id']
    u,*v=a.to_list()
    for _v in v:
        g.add_edge(map.loc[u].hash,map.loc[_v].hash)
print('done 2')
wcg=list(nx.weakly_connected_components(g))
print("numero di nodi "+str(g.number_of_nodes() ))
print("numero di archi "+str(g.number_of_edges()))
cluster_sizes=[ len(x) for x in wcg]
print(max(cluster_sizes),min(cluster_sizes),sum(cluster_sizes)/len(cluster_sizes))
cluster_stats=defaultdict(lambda:0)
for x in cluster_sizes:
    cluster_stats[x]+=1
cluster_stats
sorted(wcg,key=len,reverse=True)[0:10]

C:\Users\katyg\AppData\Local\Temp\ipykernel_10536\997782170.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for k,group in islice(tx_addresses,10000):


done 2
numero di nodi 59
numero di archi 39
7 2 2.6818181818181817


[{'124wRWVmAkRejjv6aPuF3B4qi5ww4uYKLy',
  '12higDjoCCNXSA95xZMWUdPvXNmkAduhWv',
  '12yrUmFBpsoh9CrV38LyxsAtJ4Y7ymNJ7y',
  '16VkpmfxEWKxgFsCbGmLKRLD8ARrAXnr3g',
  '18dBAAdfR23pioGd9VN1Ji1pgGoHBr8RBm',
  '1GSUxU95RifHzedGR1Fmk2z7oZ6SfUonK4',
  '1PwTyRyX3xBA1NLPuii9qBkeYv7uAouRHa'},
 {'1ACHpxEQg9hWgqFiBGu6r6g7Bgbtgrsr4p',
  '1AbHNFdKJeVL8FRZyRZoiTzG9VCmzLrtvm',
  '1CbAopr7sNMKjsJa8dAS6P5sM788kmMooo',
  '1Fmjwt8172FZT5XdKnzTUcEEzc1T2MCg2a',
  '1Hr2Pj6p984sZ8eC9333vZTkfZhASTEMPN'},
 {'12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S',
  '13HtsYzne8xVPdGDnmJX8gHgBZerAfJGEf',
  '1LzBzVqEeuQyjD2mRWHes3dgWrT9titxvq',
  '1Q2TWHE3GMdB6BZKafqwxXtWAWgFt5Jvm3'},
 {'16juSNx2hwNkGoEpDYu9s6kWjkmk8ufe13',
  '1Cpd6ijLsakiAVTRqegjJyhXe61KYjy4ro',
  '1EsHBdSHrHQ42UzoBYx6upKu9x25NFXyBP',
  '1MjJkkBtL8TTQDQ8TNULe99xwWhz214YhS'},
 {'1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi',
  '1DZTzaBHUDM7T3QvUKBz4qXMRpkg8jsfB5',
  '1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv'},
 {'12hPHVEb2W5ppJ43i2Mx8SkqebxcSt2Huj',
  '17sbu8Z9FG9PoyqrgTdQzR8yg3SW8U7H

# deanonimizzazione
